1 x 1 Convolution Filters

Can stack multiple times to create a channels = filters of any size

Also provide as a good source of dimensionality reduction

Reduce operations of large filters of a large image

Inception modules

Use a bunch of different filters and take there results and stack them into individual channels of a single image

(Average Pooling)

In [6]:
import torch as t
import torch.nn.functional as F
import torch.nn as nn

In [9]:
class InceptionA(nn.Module):
    def __init__(self, in_channels):
        super(InceptionA, self).__init__()
        self.branch1x1 = nn.Conv2d(in_channels, 16, kernel_size=1) # 16 layers of 1 x 1 random convolutions
        
        self.branch5x5_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch5x5_2 = nn.Conv2d(16, 24, kernel_size=5, padding=2)
        
        self.branch3x3dbl_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch3x3dbl_2 = nn.Conv2d(16, 24, kernel_size=3, padding=1)
        self.branch3x3dbl_3 = nn.Conv2d(24, 24, kernel_size=3, padding=1)
        
        self.branch_pool = nn.Conv2d(in_channels, 24, kernel_size=1) # 24 layers of 1 x 1 random convolutions

    def forward(self, x):
        branch1x1 = self.branch1x1(x)
            
        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)
        
        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)
        
        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)
        
        return torch.cat([branch1x1, branch5x5, branch3x3dbl, branch_pool], 1)

In [10]:
import torch.nn.functional as F

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv_1 = t.nn.Conv2d(1, 10, kernel_size=5)
        self.conv_2 = t.nn.Conv2d(88, 20, kernel_size=5)
        
        self.incept_1 = InceptionA(10)
        self.incept_2 = InceptionA(20)
    
        self.mp = t.nn.MaxPool2d(2)
        self.fc = t.nn.Linear(1408, 10)
        
    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv_1(x)))
        x = self.incept_1(x)
        x = F.relu(self.mp(self.conv_2(x)))
        x = self.incept_2(x)
        x = x.view(in_size, -1) # flatten 1d
        x = self.fc(x)
        return F.log_softmax(x)

In [12]:
from torchvision import datasets, transforms

# Training settings
batch_size = 64

# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./mnist_data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = t.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = t.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [13]:
model = Net()
criterion = t.nn.CrossEntropyLoss()
optimizer = t.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        output = model.forward(data)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

def test():
    model.eval() # put in eval mode, no gradients
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).data[0]
        # get the index of the max
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


for epoch in range(1, 10):
    train(epoch)
    test()

/Users/achang/miniconda2/envs/deep-learning-lab/lib/python3.6/site-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/Users/achang/miniconda2/envs/deep-learning-lab/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  app.launch_new_instance()


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.303167
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.300485
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.299089
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.288016
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.307461
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.295238
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.288796
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.304894
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.309413
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.300480
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.294132
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.287025
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.292865
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.302984
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.268027
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.274787
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.284865
Train Epoch: 1 [10880/60000 (18%)]	Loss: 2.282295
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.263808
Train Epoch: 1 [12160/60000 (20%)]	Loss: 2.268842
Train Epoch: 1 [12800/60000 (

/Users/achang/miniconda2/envs/deep-learning-lab/lib/python3.6/site-packages/ipykernel_launcher.py:25: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number



Test set: Average loss: 0.0026, Accuracy: 9433/10000 (94%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.156893
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.085559
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.175403
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.183048
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.047647
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.230784
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.123615
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.087302
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.089346
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.096888
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.125713
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.093574
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.345931
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.138214
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.141641
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.082938
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.392035
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.193461
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.154589
Train Epoch: 2 [12

Train Epoch: 3 [44160/60000 (74%)]	Loss: 0.055505
Train Epoch: 3 [44800/60000 (75%)]	Loss: 0.021661
Train Epoch: 3 [45440/60000 (76%)]	Loss: 0.089394
Train Epoch: 3 [46080/60000 (77%)]	Loss: 0.061161
Train Epoch: 3 [46720/60000 (78%)]	Loss: 0.022970
Train Epoch: 3 [47360/60000 (79%)]	Loss: 0.023171
Train Epoch: 3 [48000/60000 (80%)]	Loss: 0.032649
Train Epoch: 3 [48640/60000 (81%)]	Loss: 0.095364
Train Epoch: 3 [49280/60000 (82%)]	Loss: 0.078275
Train Epoch: 3 [49920/60000 (83%)]	Loss: 0.055142
Train Epoch: 3 [50560/60000 (84%)]	Loss: 0.048644
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.062989
Train Epoch: 3 [51840/60000 (86%)]	Loss: 0.120235
Train Epoch: 3 [52480/60000 (87%)]	Loss: 0.047273
Train Epoch: 3 [53120/60000 (88%)]	Loss: 0.024215
Train Epoch: 3 [53760/60000 (90%)]	Loss: 0.131953
Train Epoch: 3 [54400/60000 (91%)]	Loss: 0.084116
Train Epoch: 3 [55040/60000 (92%)]	Loss: 0.044214
Train Epoch: 3 [55680/60000 (93%)]	Loss: 0.028999
Train Epoch: 3 [56320/60000 (94%)]	Loss: 0.066300


Train Epoch: 5 [28160/60000 (47%)]	Loss: 0.102387
Train Epoch: 5 [28800/60000 (48%)]	Loss: 0.024820
Train Epoch: 5 [29440/60000 (49%)]	Loss: 0.014833
Train Epoch: 5 [30080/60000 (50%)]	Loss: 0.097017
Train Epoch: 5 [30720/60000 (51%)]	Loss: 0.037964
Train Epoch: 5 [31360/60000 (52%)]	Loss: 0.041912
Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.096828
Train Epoch: 5 [32640/60000 (54%)]	Loss: 0.014743
Train Epoch: 5 [33280/60000 (55%)]	Loss: 0.039356
Train Epoch: 5 [33920/60000 (57%)]	Loss: 0.138621
Train Epoch: 5 [34560/60000 (58%)]	Loss: 0.003923
Train Epoch: 5 [35200/60000 (59%)]	Loss: 0.103865
Train Epoch: 5 [35840/60000 (60%)]	Loss: 0.033082
Train Epoch: 5 [36480/60000 (61%)]	Loss: 0.159305
Train Epoch: 5 [37120/60000 (62%)]	Loss: 0.084629
Train Epoch: 5 [37760/60000 (63%)]	Loss: 0.064038
Train Epoch: 5 [38400/60000 (64%)]	Loss: 0.113145
Train Epoch: 5 [39040/60000 (65%)]	Loss: 0.152215
Train Epoch: 5 [39680/60000 (66%)]	Loss: 0.161193
Train Epoch: 5 [40320/60000 (67%)]	Loss: 0.031716


Train Epoch: 7 [12160/60000 (20%)]	Loss: 0.021335
Train Epoch: 7 [12800/60000 (21%)]	Loss: 0.122996
Train Epoch: 7 [13440/60000 (22%)]	Loss: 0.116458
Train Epoch: 7 [14080/60000 (23%)]	Loss: 0.109213
Train Epoch: 7 [14720/60000 (25%)]	Loss: 0.039515
Train Epoch: 7 [15360/60000 (26%)]	Loss: 0.192098
Train Epoch: 7 [16000/60000 (27%)]	Loss: 0.082182
Train Epoch: 7 [16640/60000 (28%)]	Loss: 0.035756
Train Epoch: 7 [17280/60000 (29%)]	Loss: 0.031194
Train Epoch: 7 [17920/60000 (30%)]	Loss: 0.018114
Train Epoch: 7 [18560/60000 (31%)]	Loss: 0.084972
Train Epoch: 7 [19200/60000 (32%)]	Loss: 0.020271
Train Epoch: 7 [19840/60000 (33%)]	Loss: 0.046859
Train Epoch: 7 [20480/60000 (34%)]	Loss: 0.052168
Train Epoch: 7 [21120/60000 (35%)]	Loss: 0.025011
Train Epoch: 7 [21760/60000 (36%)]	Loss: 0.014196
Train Epoch: 7 [22400/60000 (37%)]	Loss: 0.018238
Train Epoch: 7 [23040/60000 (38%)]	Loss: 0.022706
Train Epoch: 7 [23680/60000 (39%)]	Loss: 0.049291
Train Epoch: 7 [24320/60000 (41%)]	Loss: 0.033584


Train Epoch: 8 [56960/60000 (95%)]	Loss: 0.074238
Train Epoch: 8 [57600/60000 (96%)]	Loss: 0.009824
Train Epoch: 8 [58240/60000 (97%)]	Loss: 0.123143
Train Epoch: 8 [58880/60000 (98%)]	Loss: 0.141791
Train Epoch: 8 [59520/60000 (99%)]	Loss: 0.016634

Test set: Average loss: 0.0008, Accuracy: 9833/10000 (98%)

Train Epoch: 9 [0/60000 (0%)]	Loss: 0.072020
Train Epoch: 9 [640/60000 (1%)]	Loss: 0.052810
Train Epoch: 9 [1280/60000 (2%)]	Loss: 0.258126
Train Epoch: 9 [1920/60000 (3%)]	Loss: 0.025870
Train Epoch: 9 [2560/60000 (4%)]	Loss: 0.022012
Train Epoch: 9 [3200/60000 (5%)]	Loss: 0.075332
Train Epoch: 9 [3840/60000 (6%)]	Loss: 0.040695
Train Epoch: 9 [4480/60000 (7%)]	Loss: 0.024811
Train Epoch: 9 [5120/60000 (9%)]	Loss: 0.021174
Train Epoch: 9 [5760/60000 (10%)]	Loss: 0.018023
Train Epoch: 9 [6400/60000 (11%)]	Loss: 0.021207
Train Epoch: 9 [7040/60000 (12%)]	Loss: 0.154908
Train Epoch: 9 [7680/60000 (13%)]	Loss: 0.010209
Train Epoch: 9 [8320/60000 (14%)]	Loss: 0.072699
Train Epoch: 9 [

Cannot always do infinite layers, eventually training error goes higher
* Vanishing gradients problem
* Back propagation kind of gives up
* Degradation problem
* Network accuracy gets saturated (weights) and degrades

How do deal with previous problems?

* Deep Residual Learning
- Add image on-top of module
- ResNet
- DenseNet
* Add previous module to next module
* Network Design

In [16]:
t.save(model, './mnist_advanced_cnn_model')

/Users/achang/miniconda2/envs/deep-learning-lab/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/Users/achang/miniconda2/envs/deep-learning-lab/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type InceptionA. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [18]:
new_model = t.load('./mnist_advanced_cnn_model')
new_model.state_dict()

OrderedDict([('conv_1.weight',
              tensor([[[[ 0.3554,  0.1085,  0.0121, -0.2704, -0.1879],
                        [ 0.0815, -0.0032, -0.1400,  0.2563,  0.1377],
                        [ 0.1817,  0.3597,  0.2510,  0.1289,  0.2135],
                        [ 0.2649,  0.3838,  0.1845,  0.1953,  0.2509],
                        [ 0.0657, -0.2292, -0.1969, -0.1881, -0.2648]]],
              
              
                      [[[ 0.0044, -0.1581,  0.1665,  0.1556,  0.0801],
                        [-0.0255, -0.2009,  0.1600, -0.1494, -0.1635],
                        [-0.1675, -0.0667, -0.1957,  0.0574, -0.1416],
                        [ 0.0049, -0.1766,  0.1048,  0.1532, -0.1138],
                        [ 0.0074,  0.1699,  0.0365,  0.1589, -0.0628]]],
              
              
                      [[[ 0.0165, -0.0451,  0.0076, -0.1666, -0.0893],
                        [ 0.1528, -0.0476,  0.0008, -0.3004, -0.3275],
                        [ 0.0385,  0.0321, -0.0734,  

## ToDo 11-1 / 11-2 / 113